In [35]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
messages = []

In [36]:
import requests
base_url = "https://nomad-movies.nomadcoders.workers.dev"

def get_popular_movies():
    # return ["The Shawshank Redemption", "The Godfather", "The Dark Knight"]
    response = requests.get(f"{base_url}/movies")
    return response.json()

def get_movie_details(id):
    # return {"id": id, "title": "The Shawshank Redemption", "year": 1994}
    response = requests.get(f"{base_url}/movies/{id}")
    return response.json()

def get_similar_movies(id):
    # return ["The Green Mile", "Forrest Gump", "The Silence of the Lambs"]
    response = requests.get(f"{base_url}/movies/{id}/similar")
    return response.json()

FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_similar_movies": get_similar_movies,
}

In [37]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "인기있는 영화 목록 조회 (응답에 영화 제목과 아이디(id) 반드시 포함)",
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "영화 아이디(id)로 상세 조회",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "integer",
                        "description": "영화 아이디(id)",
                    }
                },
                "required": ["id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "영화 아이디(id)로 유사한 영화 목록 조회",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "integer",
                        "description": "영화 아이디(id)",
                    }
                },
                "required": ["id"],
            },
        },
    },
]

In [31]:
import json
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):

    if message.tool_calls:
        # messages.append(message)
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name

            try:
                arguments = json.loads(tool_call.function.arguments)
            except json.JSONDecodeError:
                arguments = {}

            print(f"🛠️ 시스템: {function_name}({arguments}) 실행 중...")

            function_to_run = FUNCTION_MAP.get(function_name)
            if function_to_run:
                try:
                    raw_result = function_to_run(**arguments)
                    result_content = json.dumps(raw_result, ensure_ascii=False)
                except Exception as e:
                    result_content = f"Error: {str(e)}"
            else:
                result_content = "Function not found."

            # print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result_content,
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"Agent: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message) 


In [38]:
while True:
    message = input("Send a message to the Movie ChatBot...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: 인기있는 영화를 좀 알려줘
🛠️ 시스템: get_popular_movies({}) 실행 중...
Agent: 여기 인기 있는 영화 목록입니다:

1. **Shelter**
   - **개요**: 한 남자가 폭풍에서 어린 소녀를 구한 후, 과거의 적들로부터 그녀를 보호하기 위해 고립된 삶에서 벗어나는 이야기를 그립니다.
   - **개봉일**: 2026-01-28
   - **평점**: 6.8
   - ![Shelter](https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg)

2. **The Bluff**
   - **개요**: 평화로운 섬에서의 삶이 복수심에 불타는 과거의 선장이 돌아오면서 시작되는 이야기를 다룹니다.
   - **개봉일**: 2026-02-17
   - **평점**: 5.995
   - ![The Bluff](https://image.tmdb.org/t/p/w780/sojEzvfxR2DBcDSJyAisX8TWjov.jpg)

3. **Mercy**
   - **개요**: 미래의 판사 AI에게 아내 살인 혐의로 재판받는 형사가 자신을 변호하기 위해 싸우는 이야기입니다.
   - **개봉일**: 2026-01-20
   - **평점**: 7.149
   - ![Mercy](https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg)

4. **Pose**
   - **개요**: 한 고독한 예술가가 그의 과거와 맞닥뜨리며 조카와 한 주말을 보내는 긴장된 이야기를 나눕니다.
   - **개봉일**: 2026-02-25
   - **평점**: 8.0
   - ![Pose](https://image.tmdb.org/t/p/w780/qhyyyWrHUbl6QG4udAJj17CBa5.jpg)

5. **Hellfire**
   - **개요**: 신비로운 과거를 가진 방랑자가 작은 마을에 도착해 범죄 보스와 맞서 싸웁니다.
   - 